<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/RotinaDeTreinamentoModelosDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rotinas de Treinamento


## Configurações Iniciais 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install keras==2.7.0
! pip show keras

In [ ]:
import pandas as pd
import numpy as np
import string, re
import itertools
import nltk
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.callbacks import EarlyStopping
py.init_notebook_mode(connected=True)
%matplotlib inline
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

## Lendo a Base

In [ ]:
# CAMINHO DOS DADOS

#Base1 Com StopWords
PATH_BASE1 = "/content/drive/MyDrive/TCC/dados/processadas/Base1_classificada.csv"
#Base2 Sem StopWords
PATH_BASE2 = "/content/drive/MyDrive/TCC/dados/processadas/Base2_classificada.csv"
#teste Leila
PATH_BASE3 = "/content/drive/MyDrive/TCC/dados/processadas/proc_NB.csv"


In [ ]:
trainDF = pd.read_csv(PATH_BASE3, index_col=0)
print(trainDF)

          id                                               text        label
0          0                             igoryarlei voce também  not_hatetag
1          1                       ambiguinho eu estou com nojo      hatetag
2          2                           geral foi no jogo do foz  not_hatetag
3          3                  pois é né gente é isso ai pessoal  not_hatetag
4          4  um dj que eu bato no peito e falo que é o irmã...  not_hatetag
...      ...                                                ...          ...
34085  42987    ain que nordestino não sabe votar e blá blá blá      hatetag
34086  42988   marcelo_emanoel imagine essa turma da extrema...      hatetag
34087  42989  ei seu coiso tomasse o que pra ficar com essa ...      hatetag
34088  42990  magno malta evangélico e maior disseminador de...      hatetag
34089  42991   fatosnacionais será que os gritos de democrac...      hatetag

[32510 rows x 3 columns]


In [ ]:
# Dropa colunas que não serão utilizadas
trainDF.drop(['id_str', 'created_at','text','entities.hashtags','place.full_name','classificacao_badword','text_limpo'], axis='columns', inplace=True)

# Renomeia as colunas que sobraram
trainDF.rename(columns = {'text_cleanStopWords': 'text', 'classificacao_hate': 'label'}, inplace = True)
trainDF.head(5)

In [ ]:
print(trainDF['text'])

0                                   igoryarlei voce também
1                             ambiguinho eu estou com nojo
2                                 geral foi no jogo do foz
3                        pois é né gente é isso ai pessoal
4        um dj que eu bato no peito e falo que é o irmã...
                               ...                        
34085      ain que nordestino não sabe votar e blá blá blá
34086     marcelo_emanoel imagine essa turma da extrema...
34087    ei seu coiso tomasse o que pra ficar com essa ...
34088    magno malta evangélico e maior disseminador de...
34089     fatosnacionais será que os gritos de democrac...
Name: text, Length: 32510, dtype: object


In [ ]:
# Pega a média de caracteres dos tweets de toda a base
average_len = trainDF['text'].apply(lambda x: len(str(x))).mean()
average_len

61.45521378037527

In [ ]:
# Agrupa dados por label
trainDF.groupby('label').count()

,id,text
label,,
hatetag,4124,4124
not_hatetag,28386,28386


In [ ]:
# convert output  hate =1 not_hate =0
# trainDF.label = [1 if each == "hate" else 0 for each in trainDF.label] 
trainDF.label = [1 if each == "hatetag" else 0 for each in trainDF.label] 

In [ ]:
categorias_desejadas = [0, 1] # !!!
df_filtrado = trainDF[trainDF.label.isin(categorias_desejadas)]
df_filtrado.shape

(32510, 3)

In [ ]:
df_filtrado

,id,text,label
0,0,igoryarlei voce também,0
1,1,ambiguinho eu estou com nojo,1
2,2,geral foi no jogo do foz,0
3,3,pois é né gente é isso ai pessoal,0
4,4,um dj que eu bato no peito e falo que é o irmã...,0
...,...,...,...
34085,42987,ain que nordestino não sabe votar e blá blá blá,1
34086,42988,marcelo_emanoel imagine essa turma da extrema...,1
34087,42989,ei seu coiso tomasse o que pra ficar com essa ...,1
34088,42990,magno malta evangélico e maior disseminador de...,1


In [ ]:
def balanceia_dados(df, verbose=0):
    # Esta funcao recebe um dataframe original e retorna um dataset
    # contendo um numero igual de noticias das categorias
    # por simplicidade, vou supor que são somente duas categorias.
    global categorias_desejadas
    df_balanceado = df.copy()
    
    # cria um dataframe para cada classe. Assim podemos remover o excesso
    df_F = df_balanceado.loc[df_balanceado.label == categorias_desejadas[0]]
    df_T = df_balanceado.loc[df_balanceado.label == categorias_desejadas[1]]

    num_amostras_F, num_amostras_T = df_F.shape[0], df_T.shape[0]
    num_amostras_a_remover = abs(num_amostras_F - num_amostras_T)
    
    # remove amostras da classe majoritaria
    if num_amostras_F > num_amostras_T:
        #print(df_F[:num_amostras_a_remover)
        df_F = df_F[num_amostras_a_remover:]
        
    elif num_amostras_T > num_amostras_F:
        #
        df_T = df_T[num_amostras_a_remover:]

    return pd.concat([df_F, df_T])

In [ ]:
# Aplica função de balancear os dados
dataBC = balanceia_dados(df_filtrado)

# DATA BALANCEADO
trainDF = dataBC
trainDF.reset_index(drop=True, inplace=True) # reseta index
trainDF.shape

(8248, 3)

In [ ]:
trainDF.groupby('label').count()

,id,text
label,,
0,4124,4124
1,4124,4124


In [ ]:
trainDF['label']

0       0
1       0
2       0
3       0
4       0
       ..
8243    1
8244    1
8245    1
8246    1
8247    1
Name: label, Length: 8248, dtype: int64

In [ ]:
# Salva os registros da coluna 'text' do dataframe em uma lista
text = []
trainDF['text'].apply(lambda x: text.append(x))
print(text)

[' rosa_de_luta e globalista sorista', ' joaovictordias ai elogio grandão hein', ' the__clau se fosse minha pic', 'são homens como esses que fazem eu sentir vergonha de ser um', ' imozartts os shows foram sold out em minutos', ' _larisexy_ que linda', 'após 2 apresentações pra lá de especiais nas últimas edições da elrow no brasil em são paulo e no green valley o fancy inc que esta em', 'o bom de trabalhar com maquiadoras é que eu quando estou com preguiça de fazer uma make elas fazem por mim', 'não importa quem vai governar benefícios a parentes é uma coisa cultural no brasil muito difícil de mudar com rapidez mas não vai ser por isso que a sociedade vai aceitar a prática', 'estou aqui na 12', 'santa rita jacutinga vai rolar carna esse ano', 'gentevamos fazer alguma coisa', 'muitas e muitas brigas mas nunca deixei de te considerar minha melhor', 'eita mundo louco que girou de novo', 'vem na transparência maldita porque se vier jogando verde nois fuma', 'a minha vontade de comprar meu 

In [ ]:
# converte average_len em inteiro
average_len = int(average_len)
print(average_len)

61


In [ ]:
# # Para que transformar os tuítes em matriz de números
# tokenizer = Tokenizer(num_words=5000, split=' ')

# tokenizer.fit_on_texts(text)
# X = tokenizer.texts_to_sequences(text)  

# # limitando o tamanho das sequências com base no tamanho máximo que especificamos na variável "max_sequence_length"
# X = pad_sequences(X, maxlen=average_len)

In [ ]:
quantidade_palavras_unicas = len( list ( trainDF['text'].str.split(' ', expand=True).stack().unique() ) )
quantidade_palavras_unicas

16666

In [ ]:
print(type(text))

<class 'list'>


In [ ]:
# word level tf-idf
max_features = quantidade_palavras_unicas
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features= max_features)
X = tfidf_vect.fit_transform(text)


# ngram level tf-idf
max_features = quantidade_palavras_unicas
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=max_features)
X_ngram = tfidf_vect_ngram.fit_transform(text)

# characters level tf-idf
max_features = quantidade_palavras_unicas
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=max_features)
X_ngram_chars = tfidf_vect_ngram_chars.fit_transform(text) 

In [ ]:
Y = trainDF['label']

In [ ]:
X = X.toarray()
# X_ngram = X_ngram.toarray()
# X_ngram_chars= X_ngram_chars.toarray()

In [ ]:
print(type(X))
print(X)

<class 'numpy.ndarray'>
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, Y, test_size=0.3, random_state=2)

In [ ]:
train_x.shape

(5773, 16647)

## NB

In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=0.3, random_state=2)
encoder = preprocessing.LabelEncoder() # !!!
train_y_enc = encoder.fit_transform(train_y)
valid_y_enc = encoder.fit_transform(valid_y)
print(valid_y_enc)

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

[1 0 1 ... 1 0 1]


In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [ ]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y_enc, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# # Naive Bayes on Word Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y_enc, xvalid_tfidf)
# print ("NB, WordLevel TF-IDF: ", accuracy)

# # Naive Bayes on Ngram Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y_enc, xvalid_tfidf_ngram)
# print ("NB, N-Gram Vectors: ", accuracy)

# # Naive Bayes on Character Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y_enc, xvalid_tfidf_ngram_chars)
# print ("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.8690909090909091


## SHALLOW NET

In [ ]:


def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(1000, activation="softmax")(input_layer)
     
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)

    classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
    
    return classifier 

# Modelo criado
model = create_model_architecture(train_x.shape[1])

# Modelo sendo treinado/executado
epochs = 20
batch_size = 64
hist = model.fit(
      train_x, 
      train_y, 
      validation_data=(valid_x, valid_y),
      epochs=epochs,
      batch_size=batch_size, 
      shuffle=True,
      verbose=1)

Epoch 1/20
91/91 [==============================] - 16s 173ms/step - loss: 0.6931 - accuracy: 0.4985 - val_loss: 0.6930 - val_accuracy: 0.4978
Epoch 2/20
91/91 [==============================] - 15s 169ms/step - loss: 0.6929 - accuracy: 0.5112 - val_loss: 0.6928 - val_accuracy: 0.4978
Epoch 3/20
91/91 [==============================] - 15s 168ms/step - loss: 0.6926 - accuracy: 0.6425 - val_loss: 0.6923 - val_accuracy: 0.7838
Epoch 4/20
91/91 [==============================] - 17s 187ms/step - loss: 0.6917 - accuracy: 0.6321 - val_loss: 0.6913 - val_accuracy: 0.6081
Epoch 5/20
91/91 [==============================] - 15s 166ms/step - loss: 0.6900 - accuracy: 0.6167 - val_loss: 0.6892 - val_accuracy: 0.5855
Epoch 6/20
91/91 [==============================] - 15s 163ms/step - loss: 0.6863 - accuracy: 0.8358 - val_loss: 0.6848 - val_accuracy: 0.7782
Epoch 7/20
91/91 [==============================] - 15s 166ms/step - loss: 0.6793 - accuracy: 0.8373 - val_loss: 0.6770 - val_accuracy: 0.8400

In [ ]:
scores = model.evaluate(valid_x, valid_y, verbose = 0, batch_size = batch_size)
print("Acc: %.2f%%" % (scores[1]*100))

Acc: 88.20%


# Leila


In [ ]:
#!pip install tensorflow
!pip   uninstall tensorflow -y

In [ ]:
# import tensorflow as tf
# from tensorflow import keras, SparseTensor
# import pandas as pd
# import numpy as np
# import string, re
# import itertools
# import nltk
# import plotly.offline as py
# import plotly.graph_objs as go
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import accuracy_score
# #from keras.preprocessing.text import Tokenizer
# #from keras.preprocessing.sequence import pad_sequences
# #from keras.models import Sequential
# #from keras.layers import Dense, Embedding, LSTM
# #from keras.callbacks import EarlyStopping
# py.init_notebook_mode(connected=True)
# %matplotlib inline
# from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn import decomposition, ensemble
# import xgboost, textblob
# #from keras.preprocessing import text, sequence
# #from keras import layers, models, optimizers


# Divisão Trainamento/Teste






---



In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=0.25, random_state=2)

# label encode the target variable 
encoder = preprocessing.LabelEncoder() # !!!
train_y_enc = encoder.fit_transform(train_y)
valid_y_enc = encoder.fit_transform(valid_y)
print(valid_y_enc)

In [ ]:
train_y.shape

# Baseline

## NB

In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=4000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=4000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=4000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [ ]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y_enc, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y_enc, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y_enc, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y_enc, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", accuracy)

## Shallow net (tbm baseline)

In [ ]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    # opt = optimizers.Adam(learning_rate=0.01)
    opt = optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

    classifier.compile(optimizer=opt, loss='binary_crossentropy', metrics = ['acc'])
    
    return classifier 



In [ ]:
xtrain_tfidf.shape

In [ ]:
classifier = create_model_architecture(xtrain_tfidf.shape[1])
accuracyNN = train_model(classifier, xtrain_tfidf, train_y, xvalid_tfidf, is_neural_net=True)
print ("NN, Ngram Level TF IDF Vectors",  accuracyNN)

https://colab.research.google.com/drive/18ejmbJn9KVcqWJzOvAz84HxIpPMeTijU

# word embeddings

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/PUBLICAÇÕES/PUBLICAÇÕES 2020/CISTI-2020/cbow_s50_filtrado.txt')

In [ ]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('/content/drive/My Drive/PUBLICAÇÕES/PUBLICAÇÕES 2020/CISTI-2020/cbow_s50_filtrado.txt')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

In [ ]:
# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index


In [ ]:
tam= len(word_index)
tam

In [ ]:
# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=100)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=100)

In [ ]:
# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# CNN model

In [ ]:
def create_cnn():
    # Add an Input Layer
    input_layer = keras.layers.Input((100, ))

    # Add the word embedding Layer
    embedding_layer = keras.layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], trainable=True)(input_layer)
    embedding_layer = keras.layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = keras.layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = keras.layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = keras.layers.Dense(64, activation="relu")(pooling_layer)
    output_layer1 = keras.layers.Dropout(0.25)(output_layer1)
    output_layer2 = keras.layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = keras.models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=sgd, loss='mean_squared_error', metrics = ['acc', 'mae'])
    return model



In [ ]:
classifier = create_cnn()
accuracyCNN = train_model(classifier,train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("CNN, Word Embeddings - accuracy",  accuracyCNN)

# LSTM

In [ ]:
def create_rnn_lstm():
# Add an Input Layer
    input_layer = layers.Input((100, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['acc', 'mae'])
    return model



In [ ]:
classifier = create_rnn_lstm()
accuracyLSTM = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-LSTM, Word Embeddings",  accuracyLSTM)

## LSTM DO CHUCHU

In [ ]:
from sklearn import metrics
from keras import layers, models, optimizers

def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(classifier.summary())
    classifier.fit(feature_vector_train, label, epochs=3)
    return classifier

def create_rnn_lstm(embedding_matrix):
  
  model_input = layers.Input(shape=(50, ),name="input",dtype='int32')
  embedding = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix])(model_input)
  embedding = layers.SpatialDropout1D(0.3)(embedding)
  lstm = layers.LSTM(1000, dropout =0.1, name="LSTM")(embedding)
  output = layers.Dense(1, activation='sigmoid',name='sigmoid')(lstm)
  model = models.Model(inputs=model_input,outputs=output)
  return model

classifier = create_rnn_lstm(embedding_matrix)
trainedModel = train_model(classifier, train_seq_x, train_y, valid_seq_x)
# predict the labels on validation dataset
predictions = trainedModel.predict(valid_seq_x)
predictions = predictions.round()

print("RNN-LSTM, Word Embeddings",  metrics.accuracy_score(valid_y,predictions))
print(predictions)

# RNN gru

In [ ]:
def create_rnn_gru():
  # Add an Input Layer
    input_layer = layers.Input((100, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(64, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    #model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    #model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics = ['acc'])
    model.compile(optimizer=sgd, loss='mean_squared_error', metrics = ['acc','mae'])
    
    return model

In [ ]:
classifier = create_rnn_gru()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-GRU, Word Embeddings",  accuracy)





## Bidirectional RNN

In [ ]:
def create_bidirectional_rnn():
   # Add an Input Layer
    input_layer = layers.Input((150, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(64, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['acc','mae'])
    
    return model

In [ ]:
classifier = create_bidirectional_rnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-Bidirectional, Word Embeddings",  accuracy)

##  - CRNN Convolutional recurrent neural network

In [ ]:
def create_rcnn():
  # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], trainable=True)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['acc'])
    
    return model

In [ ]:
classifier = create_rcnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("CNN, Word Embeddings",  accuracy)